In [ ]:
# Install needed libraries
%pip install -U python-jobspy
%pip install tqdm
%pip install xlsxwriter

You should consider upgrading via the '/home/wagner/.pyenv/versions/3.10.0/envs/market_scrapper_venv/bin/python -m pip install --upgrade pip' command.
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
You should consider upgrading via the '/home/wagner/.pyenv/versions/3.10.0/envs/market_scrapper_venv/bin/python -m pip install --upgrade pip' command.


## Post-steps checking proxy-list

In [7]:
# %%capture
# !wget https://raw.githubusercontent.com/TheSpeedX/SOCKS-List/master/http.txt

In [8]:
# with open('http.txt', 'r') as file:
#     lines = file.readlines()
# proxies = [line.strip() for line in lines]

In [9]:
# %%capture
# import requests
# import threading

# working_proxies = []

# def check_proxy(proxy):
#     proxies = {'http': proxy, 'https': proxy}
#     try:
#         response = requests.get('https://google.com', proxies=proxies, timeout=5)
#         if response.status_code == 200:
#             working_proxies.append(proxy)
#     except requests.exceptions.RequestException:
#         print(f'Proxy {proxy} is not working')

# threads = []
# for proxy in proxies:
#     t = threading.Thread(target=check_proxy, args=(proxy,))
#     t.start()
#     threads.append(t)

# for t in threads:
#     t.join()

## Global configuration for scrapping

In [10]:
# Import libraries
import csv
import pandas as pd
import re
from jobspy import scrape_jobs
from tqdm import tqdm
from itertools import product
from concurrent.futures import ThreadPoolExecutor, as_completed
import requests
import time
import random
import os

# Define search terms and locations
search_terms = [
    "Data Scientist", "Data Analyst", "Cloud Architect", "Cloud Engineer",
    "Software Tester", "Machine Learning Engineer", "Software Engineer", "DevOps Engineer",
    "Stagiaires", "Développeur Informatique", "Java", "Fullstack", "Backend", "Frontend",
    "Back-end", "Front-end", "Pfe", "Internship", "Développeur", "devops", "#pfe"
]

# General locations
locations = [
    "Fez, Fès-Meknès, Morocco", "Casablanca Metropolitan Area",
    "Tanger-Tetouan-Al Hoceima, Morocco", "Rabat, Rabat-Salé-Kénitra, Morocco",
    "Marrakesh, Marrakesh-Safi, Morocco", "Morocco"
]

# Glassdoor-specific locations
glassdoor_locations = [
    "Casablanca, Greater Casablanca", "Fès",
    "Rabat (Morocco)", "Tangier, Tanger-Tétouan (Morocco)", "Marrakech-Tensift-Al Haouz, Morocco"
]

#proxies = working_proxies

results_wanted = 100
hours_old = 24
job_types = ['internship', 'fulltime']

linkedin_fetch_description = False

os.makedirs('./raw_data', exist_ok=True)
os.makedirs('./processed_data', exist_ok=True)

## Source code for scrapping

In [11]:
def check_proxy(proxy):
    try:
        response = requests.get('http://google.com', proxies={
            'http': f'http://{proxy}',
            'https': f'http://{proxy}'
        }, timeout=5)
        print(f"Proxy {proxy} IP: {response.json()['origin']}")
    except Exception as e:
        print(f"Proxy {proxy} failed: {e}")

def scrape_single_job(params):
    site_name = params['site_name']
    search_term = params['search_term']
    location = params['location']
    job_type = params['job_type']
    results_wanted = params['results_wanted']
    hours_old = params['hours_old']
    proxy = params.get('proxy')  # Get the proxy from params
    extra_params = params.get('extra_params', {})

    # Map job_type to jobType column value
    job_type_mapping = {
        'internship': 'Internship',
        'fulltime': 'Full-time'
    }
    job_type_label = job_type_mapping.get(job_type, 'Unknown')

    try:
        time.sleep(random.uniform(1, 3))
        scrape_params = {
            'site_name': site_name,
            'search_term': search_term,
            'location': location,
            'results_wanted': results_wanted,
            'hours_old': hours_old,
            'job_type': job_type,
        }
        if extra_params:
            scrape_params.update(extra_params)

        # Include proxy in scrape_params if provided
        if proxy:
            check_proxy(proxy)
            scrape_params['proxies'] = [proxy]

        jobs = scrape_jobs(**scrape_params)

        if not jobs.empty:
            jobs['jobType'] = job_type_label

        return jobs
    except Exception as e:
        print(f"Error fetching jobs for '{search_term}' in '{location}' on '{site_name}': {e}")
        return pd.DataFrame()  # Return empty DataFrame on error

def scrape_site_jobs_parallel(site_name, search_terms, locations, results_wanted, hours_old, job_types, extra_params=None, proxies=None):
    results_list = []
    combinations = list(product(search_terms, locations, job_types))

    # If proxies are provided, calculate the number of proxies
    if proxies:
        num_proxies = len(proxies)
        print(f"Using {num_proxies} proxies")
    else:
        num_proxies = 0

    # Prepare parameters for each thread
    params_list = []
    for idx, (search_term, location, job_type) in enumerate(combinations):
        params = {
            'site_name': site_name,
            'search_term': search_term,
            'location': location,
            'job_type': job_type,
            'results_wanted': results_wanted,
            'hours_old': hours_old,
            'extra_params': extra_params
        }
        if proxies and num_proxies > 0:
            proxy = proxies[idx % num_proxies]  # Assign proxies in a round-robin fashion
            params['proxy'] = proxy
        params_list.append(params)

    max_workers = 3  # Adjust based on your system's capabilities

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Start the operations and mark each future with its params
        futures = {executor.submit(scrape_single_job, params): params for params in params_list}

        for future in tqdm(as_completed(futures), total=len(futures), desc=f"{site_name.capitalize()} - Progress"):
            result = future.result()
            if not result.empty:
                results_list.append(result)
            if idx < len(params_list) - 1:  # Don't sleep after the last combination
                print(f"Sleeping for 2 minutes after combination {idx + 1} of {len(params_list)}")
                time.sleep(120)  # 2 minutes sleep

    if results_list:
        combined_results = pd.concat(results_list, ignore_index=True)
    else:
        combined_results = pd.DataFrame()
    return combined_results

## Code launch

In [12]:
# LinkedIn Scraping
print("Scraping LinkedIn")
linkedin_results = scrape_site_jobs_parallel(
    site_name='linkedin',
    search_terms=search_terms,
    locations=locations,
    results_wanted=results_wanted,
    hours_old=hours_old,
    job_types=job_types,
    # proxies=proxies,
    extra_params={'linkedin_fetch_description': linkedin_fetch_description}
)

print(f"Found {len(linkedin_results)} jobs on LinkedIn")
linkedin_results.to_csv('./raw_data/jobs_linkedin.csv', quoting=csv.QUOTE_NONNUMERIC, escapechar="\\", index=False)

# Glassdoor Scraping
print("Scraping Glassdoor")
glassdoor_results = scrape_site_jobs_parallel(
    site_name='glassdoor',
    search_terms=search_terms,
    locations=glassdoor_locations,
    results_wanted=results_wanted,
    hours_old=hours_old,
    job_types=job_types
    # proxies=proxies
)

print(f"Found {len(glassdoor_results)} jobs on Glassdoor")
glassdoor_results.to_csv('./raw_data/jobs_glassdoor.csv', quoting=csv.QUOTE_NONNUMERIC, escapechar="\\", index=False)

# Indeed Scraping
print("Scraping Indeed")
indeed_results = scrape_site_jobs_parallel(
    site_name='indeed',
    search_terms=search_terms,
    locations=locations,
    results_wanted=results_wanted,
    hours_old=hours_old,
    job_types=job_types,
    # proxies=proxies,
    extra_params={'country_indeed': 'Morocco'}
)

print(f"Found {len(indeed_results)} jobs on Indeed")
indeed_results.to_csv('./raw_data/jobs_indeed.csv', quoting=csv.QUOTE_NONNUMERIC, escapechar="\\", index=False)

Scraping LinkedIn


Linkedin - Progress: 100%|██████████| 252/252 [06:24<00:00,  1.53s/it]


Found 602 jobs on LinkedIn
Scraping Glassdoor


Glassdoor - Progress:   0%|          | 0/210 [00:00<?, ?it/s]2025-11-13 18:47:57,558 - ERROR - JobSpy:Glassdoor - Glassdoor response status code 400
2025-11-13 18:47:57,559 - ERROR - JobSpy:Glassdoor - Glassdoor: location not parsed
Glassdoor - Progress:   0%|          | 1/210 [00:01<05:27,  1.57s/it]2025-11-13 18:47:57,669 - ERROR - JobSpy:Glassdoor - Glassdoor response status code 400
2025-11-13 18:47:57,670 - ERROR - JobSpy:Glassdoor - Glassdoor: location not parsed
Glassdoor - Progress:   1%|          | 2/210 [00:01<02:27,  1.41it/s]2025-11-13 18:47:58,821 - ERROR - JobSpy:Glassdoor - Glassdoor response status code 400
2025-11-13 18:47:58,822 - ERROR - JobSpy:Glassdoor - Glassdoor: location not parsed
Glassdoor - Progress:   1%|▏         | 3/210 [00:02<03:08,  1.10it/s]2025-11-13 18:47:59,397 - ERROR - JobSpy:Glassdoor - Glassdoor response status code 400
2025-11-13 18:47:59,398 - ERROR - JobSpy:Glassdoor - Glassdoor: location not parsed
Glassdoor - Progress:   2%|▏         | 4/210

Found 0 jobs on Glassdoor
Scraping Indeed


Indeed - Progress: 100%|██████████| 252/252 [03:19<00:00,  1.26it/s]

Found 102 jobs on Indeed


# Data Preprocessing

In [13]:
def process_fulltime_csv(input_file, output_file):
    if os.stat(input_file).st_size == 0:
        print(f"File {input_file} is empty. Skipping processing.")
        return
    try:
        df = pd.read_csv(input_file)
        if df.empty:
            print(f"No data to parse from file {input_file}. Skipping.")
            return
    except pd.errors.EmptyDataError:
        print(f"EmptyDataError: No columns to parse from file {input_file}. Skipping.")
        return

    full_time = df[df['jobType'] == 'Full-time']
    full_time = full_time.sort_values(by='date_posted', ascending=False)
    pattern = '|'.join(keywords)
    mask = full_time['title'].str.contains(pattern, case=False, na=False)
    filtered_df = full_time[mask]
    print(output_file + " contains ->" + str(len(filtered_df)) + " offer")
    filtered_df.to_csv('./processed_data/' + str(output_file) + '_filtered_data.csv', index=False)

def process_internship_csv(input_file, output_file):
    if os.stat(input_file).st_size == 0:
        print(f"File {input_file} is empty. Skipping processing.")
        return
    try:
        df = pd.read_csv(input_file)
        if df.empty:
            print(f"No data to parse from file {input_file}. Skipping.")
            return
    except pd.errors.EmptyDataError:
        print(f"EmptyDataError: No columns to parse from file {input_file}. Skipping.")
        return

    internships = df[df['jobType'] == 'Internship']
    internships = internships.sort_values(by='date_posted', ascending=False)
    print(output_file + " contains ->" + str(len(internships)) + " offer")
    internships.to_csv('./processed_data/' + str(output_file) + '_filtered_data_interns.csv', index=False)

In [14]:
keywords = ["Stagiaire", "Développeur", "Informatique", "pfe", "intern", "Developpeur"]

for csv_file in os.listdir('./raw_data'):
    path = os.path.join('./raw_data', csv_file)
    matches = re.findall(r'jobs_(.*?)\.csv', csv_file)
    if matches:
        process_fulltime_csv(path, matches[0])
        process_internship_csv(path, matches[0])

EmptyDataError: No columns to parse from file ./raw_data/jobs_glassdoor.csv. Skipping.
EmptyDataError: No columns to parse from file ./raw_data/jobs_glassdoor.csv. Skipping.
linkedin contains ->149 offer
linkedin contains ->29 offer
indeed contains ->22 offer
indeed contains ->51 offer


In [16]:
csv_directory = './processed_data'

output_excel_file = 'interns_data.xlsx'

csv_files = [file for file in os.listdir(csv_directory) if file.endswith('.csv')]

with pd.ExcelWriter(output_excel_file, engine='xlsxwriter') as writer:
    for csv_file in csv_files:
        csv_path = os.path.join(csv_directory, csv_file)
        df = pd.read_csv(csv_path)
        sheet_name = os.path.splitext(csv_file)[0]
        df.to_excel(writer, sheet_name=sheet_name, index=False)